In [ ]:
from datetime import datetime
import math
import time
import tensorflow as tf
import os

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
batch_size = 32
num_batches = 100

In [4]:
def print_activations(t):
    print(t.op.name,' ',t.get_shape().as_list())

In [5]:
def variable_with_weigth_loss(shape, name,stddev, w1):
    var = tf.Variable(tf.truncated_normal(shape, stddev = stddev),name=name)
    if w1 is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var), w1, name = 'weight_loss')
        tf.add_to_collection('losses', weight_loss)
    return var

 * scope 内命名variable 则variabel会被自动命名为 scope_name/***, 便于区分不同层间的组件

In [6]:
def inference(images):
    parameters = []
    
    with tf.name_scope('conv1') as scope:
        kernel = tf.Variable(tf.truncated_normal([11, 11, 3, 64],dtype=tf.float32, stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(images, kernel, [1, 4, 4, 1], padding = 'SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name=scope)
        print_activations(conv1)
        parameters += [kernel, biases]
        
    with tf.name_scope('lrn1') as scope:
        lrn1 = tf.nn.lrn(conv1, depth_radius=4, bias=1.0, alpha=0.001/9, beta = 0.75, name = 'lrn1')
    pool1 = tf.nn.max_pool(lrn1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool1')
    print_activations(pool1)
    
    with tf.name_scope('conv2') as scope:
        kernel  = tf.Variable(tf.truncated_normal([5, 5, 64, 192], dtype=tf.float32, stddev=1e-1, name='weights'))
        conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[192], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(bias, name=scope) 
        parameters += [kernel, biases]
    print_activations(conv2)
    
    with tf.name_scope('lrn2') as scope:
        lrn2= tf.nn.lrn(conv2, depth_radius=4, bias=1.0, alpha=0.001/9, beta=0.75, name='lrn2')
    pool2 = tf.nn.max_pool(lrn2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool2')
    print_activations(pool2)
    
    
    with tf.name_scope('conv3') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 192, 384], dtype=tf.float32, stddev=1e-1, name='weights'))
        conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[384], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv3 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
    print_activations(conv3)
    
    with tf.name_scope('conv4') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 384, 256], dtype=tf.float32, stddev=1e-1, name='weights'))
        conv = tf.nn.conv2d(conv3, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv4 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
    print_activations(conv4)  
    
    with tf.name_scope('conv5') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32, stddev=1e-1, name='weights'))
        conv = tf.nn.conv2d(conv4, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv5 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
    print_activations(conv5)  
    pool5  = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool5')
    print_activations(pool5)
    
    with tf.name_scope('fully1') as scope:
        reshape = tf.reshape(conv5, [batch_size, -1])
        dim = reshape.get_shape()[1].value
        weight = variable_with_weigth_loss(shape=[dim,4096], name='weights',stddev=0.04, w1=0.004)
        bias = tf.Variable(tf.constant(0.1, shape=[4096]))
        fully1 = tf.nn.relu(tf.matmul(reshape, weight) + bias)
        parameters += [weight, bias]
    
    print_activations(fully1)
    with tf.name_scope('fully2') as scope:
        weight = variable_with_weigth_loss(shape=[4096,4096], name='weights',stddev=0.04, w1=0.004)
        bias = tf.Variable(tf.constant(0.1, shape=[4096]))
        fully2 = tf.nn.relu(tf.matmul(fully1, weight) + bias)
        parameters += [weight, bias]
    print_activations(fully2)  
    
    
    with tf.name_scope('fully3') as scope:
        weight = variable_with_weigth_loss(shape=[4096,1000], name='weights',stddev=0.04, w1=0.004)
        bias = tf.Variable(tf.constant(0.1, shape=[1000]))
        fully3 = tf.nn.relu(tf.matmul(fully2, weight) + bias)
        parameters += [weight, bias]
    print_activations(fully3)  

    return fully3, parameters

In [7]:
def time_tensorflow_run(session, target, info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    for i in range(num_batches + num_steps_burn_in):
        start_time =time.time()
        _ = session.run(target)
        duration = time.time() - start_time;
        if i >= num_steps_burn_in:
            if not i % 10 :
                print('%s: step%d, duration = %.3f' % (datetime.now(), i - num_steps_burn_in, duration))
            total_duration += duration;
            total_duration_squared += duration*duration
        mn = total_duration / num_batches
        vr = total_duration_squared / num_batches - mn* mn
        sd = math.sqrt(vr)
        print('$%s : %s across %d steps, %.3f +/- %.3f sec /batch' % (datetime.now(), info_string, num_batches, mn, sd))

In [8]:
def loss(logits,labels):
    labels = tf.cast(labels, tf.float32)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, 
                                                                   labels = labels, 
                                                                   name = 'cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name = 'cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'), name = 'total_loss')

In [ ]:
def run_benchmark():
    with tf.Graph().as_default():
        image_size = 224
        images = tf.Variable(tf.random_normal([batch_size, image_size, image_size, 3], dtype=tf.float32, stddev=1e-1))
        fully3, parameters = inference(images)
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        
        time_tensorflow_run(sess, fully3, "Forward")
        
        objective = tf.nn.l2_loss(fully3)
        grad = tf.gradients(objective, parameters)
        time_tensorflow_run(sess, grad, "Forward-backward")

In [ ]:
run_benchmark()

conv1   [32, 56, 56, 64]
pool1   [32, 27, 27, 64]
conv2   [32, 27, 27, 192]
pool2   [32, 13, 13, 192]
conv3   [32, 13, 13, 384]
conv4   [32, 13, 13, 256]
conv5   [32, 13, 13, 256]
pool5   [32, 6, 6, 256]
fully1/Relu   [32, 4096]
fully2/Relu   [32, 4096]
fully3/Relu   [32, 1000]
$2018-11-10 19:37:34.785199 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-10 19:37:34.804978 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-10 19:37:34.824511 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-10 19:37:34.843982 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-10 19:37:34.863581 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-10 19:37:34.883197 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-10 19:37:34.902981 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-10 19:37:34.922325 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-10 19:37:34.942151 : Forward across 100 steps, 0

$2018-11-10 19:37:36.681373 : Forward across 100 steps, 0.017 +/- 0.006 sec /batch
2018-11-10 19:37:36.700241: step90, duration = 0.019
$2018-11-10 19:37:36.700290 : Forward across 100 steps, 0.017 +/- 0.005 sec /batch
$2018-11-10 19:37:36.719287 : Forward across 100 steps, 0.017 +/- 0.005 sec /batch
$2018-11-10 19:37:36.738329 : Forward across 100 steps, 0.018 +/- 0.005 sec /batch
$2018-11-10 19:37:36.757294 : Forward across 100 steps, 0.018 +/- 0.005 sec /batch
$2018-11-10 19:37:36.776150 : Forward across 100 steps, 0.018 +/- 0.004 sec /batch
$2018-11-10 19:37:36.795281 : Forward across 100 steps, 0.018 +/- 0.004 sec /batch
$2018-11-10 19:37:36.814098 : Forward across 100 steps, 0.018 +/- 0.003 sec /batch
$2018-11-10 19:37:36.832926 : Forward across 100 steps, 0.019 +/- 0.003 sec /batch
$2018-11-10 19:37:36.851755 : Forward across 100 steps, 0.019 +/- 0.002 sec /batch
$2018-11-10 19:37:36.870873 : Forward across 100 steps, 0.019 +/- 0.000 sec /batch
$2018-11-10 19:37:37.393874 : Forw